In [2]:
import torch 
import torch.nn as nn
import numpy as np
from random import random
import matplotlib.pyplot as plt

from cartpole_env import CartPoleEnv
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

In [3]:
lr = 0.01
gamma = 1.0
num_episodes = 1000

policy = nn.Sequential(
    nn.Linear(4, 20),
    nn.LeakyReLU(),
    nn.Linear(20,2),
    nn.Softmax(dim=-1)
)

class Representation(nn.Module):
    def __init__(self):
        super().__init__()
        self.representation = nn.Sequential( 
            nn.Linear(4, 50),
            nn.LeakyReLU(),
            nn.Linear(50,20),
            nn.LeakyReLU()
        )

    def forward(self, x):
        return self.representation(x)
    
class Prediction(nn.Module):
    def __init__(self):
        super().__init__()
        self.policy = nn.Sequential(
            nn.Linear(20, 2),
            nn.Softmax(dim=-1)        
        )
        self.value = nn.Linear(20, 1)
        
    def forward(self, x):
        policy_estimate = self.policy(x)
        value_estimate = self.value(x)
        return policy_estimate, value_estimate
        
        
class Dynamics(nn.Module):
    def __init__(self):
        super().__init__()
        self.new_hidden = nn.Sequential(
            nn.Linear(20, 50),
            nn.LeakyReLU(),
            nn.Linear(50,20),
            nn.LeakyReLU()
        )
        self.reward = nn.Linear(20, 1)
    
    def forward(self, x):
        new_hidden_estimate = self.new_hidden(x)
        reward_estimate = self.reward(x)
        return new_hidden_estimate, reward_estimate
    
    

representation = Representation() # from observation to hidden
dynamics = Dynamics() # from hidden to hidden + reward
prediction = Prediction() # from hidden to policy + value


optimizer_representation = torch.optim.Adam(representation.parameters(), lr=lr)
optimizer_dynamics = torch.optim.Adam(dynamics.parameters(), lr=lr)
optimizer_prediction = torch.optim.Adam(prediction.parameters(), lr=lr)

In [ ]:
env = CartPoleEnv()
erm = EpisodeReplayMemory(gamma=gamma, lamda=1)
num_episodes = 4000
